## Blender Bot 400M Distill

In [13]:
import typing as tp
import numpy as np
import pandas as pd
import os

from collections import defaultdict
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [19]:
from transformers import BlenderbotTokenizer, BlenderbotModel, BlenderbotForConditionalGeneration

mname = 'facebook/blenderbot-400M-distill'

tokenizer = BlenderbotTokenizer.from_pretrained(mname)

generation_model = BlenderbotForConditionalGeneration.from_pretrained(mname)
model = BlenderbotModel.from_pretrained(mname)

## Run BlenderbotForConditionalGeneration
text = "My friends are cool but they eat too many carbs."
inputs = tokenizer([text], return_tensors='pt')
reply_ids = generation_model.generate(**inputs)

print(tokenizer.batch_decode(reply_ids, skip_special_tokens=True))


## Run Plain Blenderbot Model
input_ids = tokenizer("Studies have been shown that owning a dog is good for you", return_tensors="pt").input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)

last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape

Some weights of the model checkpoint at facebook/blenderbot-400M-distill were not used when initializing BlenderbotModel: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing BlenderbotModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BlenderbotModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[" That's unfortunate. Are they trying to lose weight or are they just trying to be healthier?"]


torch.Size([1, 6, 1280])

In [31]:
data.sample(5)

,utterance,name,h,m,s
254,Whoa. Like if you want to swap from Windows 95.,Joe Rogan,0,34,41
8,"That’s hilarious, too.",Joe Rogan,0,0,26
768,Oh great. Glad you like it.,Elon Musk,1,54,15
805,"Oh, I’ve read about that. Explain that. Like w...",Joe Rogan,1,56,44
376,"What does that mean to you, when you say AI Ex...",Joe Rogan,0,56,19


In [20]:
data = pd.read_csv('/home/iris/paulshab/ParlAI/data/joe_rogan_c/Joe Rogan Elon Musk Podcast Transcript May 7, 2020.csv', index_col=None)

data.head()

,utterance,name,h,m,s
0,Welcome back.,Joe Rogan,0,0,0
1,Here we go again.,Elon Musk,0,0,2
2,Great to see you and congratulations.,Joe Rogan,0,0,2
3,Thank you.,Elon Musk,0,0,4
4,You will never forget what is going on in the ...,Joe Rogan,0,0,5


## Load Dataset

In [47]:
class PadSequence:
    def __init__(self, padded_columns, device='cuda:1'):
        self.padded_columns = set(padded_columns)
        self.device = device

    def __call__(self, batch):
        padded_batch = defaultdict(list)
        for example in batch:
            for key, tensor in example.items():
                padded_batch[key].append(tensor)
            
        #print("done")       
        for key, val in padded_batch.items():
            #print(val)
            if key in self.padded_columns:
                padded_batch[key] = torch.nn.utils.rnn.pad_sequence(val, batch_first=True).to(self.device)
        return padded_batch
    

def apply_mask(example: tp.Dict[str, tp.Any], device='cuda:0', percent=0.15):
    
    # copy real input_ids:
    example['labels'] = example['input_ids'].detach().clone()
    
    # create random array of floats in equal dimension to input_ids
    rand = torch.rand(example['input_ids'].shape).to(device)

    # where the random array is less than 0.15, we set true
    mask_arr = rand < percent *  (example['input_ids'] != 101) * (example['input_ids'] != 102) * (example['input_ids'] != 0) 
    
    # create selection from mask_arr
    selection = mask_arr.nonzero().tolist()

    for i, j in selection:
        example['input_ids'][i][j] = 103
        
    return example

class Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, lines):
        self.tokenizer = tokenizer
        self.lines = lines
       
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.lines)

    def __getitem__(self, index):
        'Generates one sample of data'
        
        sample = self.tokenizer(self.lines[index])
        sample['input_ids'] = torch.as_tensor(sample['input_ids'])
        sample['token_type_ids'] = torch.tensor(sample['token_type_ids'])
        sample['attention_mask'] = torch.tensor(sample['attention_mask'])
        sample['text'] = self.lines[index]
        
        return sample
    

In [48]:
rogen_data = Dataset(tokenizer, data)

In [50]:
rogen_data[0]

KeyError: 0

In [45]:
list(kk['input_ids'])

[tensor([6950,    2])]

In [46]:
kk = tokenizer("Hello", add_special_tokens=True)
print(kk)
tokenizer.decode(kk['input_ids'])

{'input_ids': [6950, 2], 'attention_mask': [1, 1]}


' Hello</s>'

In [ ]:
rogen_data = Dataset(tokenizer, )